In [1]:
import os
import re
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from scipy.cluster.hierarchy import dendrogram
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
# Modules from scikit-learn
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

In [2]:
import nltk  
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cdbrown0702/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/cdbrown0702/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
fake = pd.read_csv("Dataset/Fake.csv")
true = pd.read_csv("Dataset/True.csv")

In [4]:
## Looping through and adding every text to a string for True
trueCorpus = []
fakeCorpus = []
for i in true['text']:
    trueCorpus.append(i) 

for i in fake['text']:
    fakeCorpus.append(i) 

In [5]:
## Tokenizing texts with NLTK
tokenized_True = [word_tokenize(i) for i in trueCorpus]
tokenized_Fake = [word_tokenize(i) for i in fakeCorpus]

In [6]:
print("trueCorpus Length: "+ str(len(tokenized_True)))
print("fakeCorpus Length: "+ str(len(tokenized_Fake)))

trueCorpus Length: 21417
fakeCorpus Length: 23481


In [7]:
## Removing Stopwords from the Corpus
stop_words = set(stopwords.words('english'))
filteredTrue = []
filteredFake = []

for wordList in tokenized_True:
    for word in wordList:
        if word not in stop_words:
            filteredTrue.append(word)

for word in tokenized_True:
    for word in wordList:
        if word not in stop_words:
            filteredFake.append(word)

In [11]:
##Removing Punctuation 

tempTrue  = ' '.join([str(elem) for elem in filteredTrue])
tokenizer = nltk.RegexpTokenizer(r"\w+")
filteredTrue = tokenizer.tokenize(tempTrue)


tempFalse  = ' '.join([str(elem) for elem in filteredFake])
tokenizer = nltk.RegexpTokenizer(r"\w+")
filteredFake = tokenizer.tokenize(tempFalse)

In [ ]:
# intent
intents = ['true', 'fake']

all_counts = Counter()
doc_counts = Counter()
intent_counts = {intent: Counter() for intent in intents}

art_data = []
art_intents = []

# iterate through true articles
for index, row in true.iterrows():
    art_counter = Counter()
    art_data.append({
        "title": row['title'],
        "text": row['text'],
        "subject": row['subject'],
        "date": row['date'],
        "counts": art_counter,
        "intent": intents[0]
    })
    art_intents.append(intents[0])
    
    filtTokens = []
    
    tokens = [word_tokenize(i) for i in row['text']]
    for wordList in tokens:
        for word in wordList:
            if word not in stop_words:
                filtTokens.append(word)
    tempTokens = ' '.join([str(elem) for elem in filtTokens])
    filtTokens = tokenizer.tokenize(tempTokens)
    
    art_counter.update(filtTokens)
    
    intent_counts['true'].update(art_counter)
    doc_counts.update(art_counter.keys())
    all_counts += art_counter